# Export model

You might want to run your train model using a different framework than PyTorch. For example, you might want to run it as a TensorRT model. 
To transition to a different framework, you need to same your model in a format that can be read by other frameworks. 
A common format is the `onnx` format.



In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader
from unetTracker.trackingProject import TrackingProject
from unetTracker.dataset import UNetDataset
from unetTracker.unet import Unet
from datetime import datetime
import torch
import pickle

In [2]:
project = TrackingProject(name="faceTrack",root_folder = "/home/kevin/Documents/trackingProjects/")

Project directory: /home/kevin/Documents/trackingProjects/faceTrack
Loading /home/kevin/Documents/trackingProjects/faceTrack/config.yalm
{'augmentation_HorizontalFlipProb': 0.0, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_size': [480, 640], 'unet_features': [64, 128, 256, 512], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'faceTrack', 'normalization_values': {'means': [0.5110162496566772, 0.4608974754810333, 0.4772901237010956], 'stds': [0.2727729380130768, 0.2578601539134979, 0.256255567073822]}, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (128.0, 0.0, 128.0)], 'objects': ['nose', 'chin', 'rEye', 'lEye'], 'target_radius': 10}


You want to set batch size to what you will be using.

In [7]:
TRAIN_IMAGE_DIR = os.path.join(project.dataset_dir,"train_images")
TRAIN_MASK_DIR =  os.path.join(project.dataset_dir,"train_masks")
TRAIN_COORDINATE_DIR = os.path.join(project.dataset_dir,"train_coordinates")
VAL_IMAGE_DIR = os.path.join(project.dataset_dir,"val_images")
VAL_MASK_DIR =  os.path.join(project.dataset_dir,"val_masks")
VAL_COORDINATE_DIR = os.path.join(project.dataset_dir,"val_coordinates")

DEVICE = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")) 
BATCH_SIZE = 2
NUM_WORKERS = 2
PIN_MEMORY=False

fileName = os.path.join(project.augmentation_dir,"valTransform")
valTransform=pickle.load(open(fileName, "rb" ))


In [8]:
valDataset = UNetDataset(VAL_IMAGE_DIR, VAL_MASK_DIR,VAL_COORDINATE_DIR, transform=valTransform) 
trainDataset = UNetDataset(TRAIN_IMAGE_DIR, TRAIN_MASK_DIR,TRAIN_COORDINATE_DIR, transform=valTransform) # use same transform as for val
dataset = UNetDataset(VAL_IMAGE_DIR, VAL_MASK_DIR,VAL_COORDINATE_DIR, transform=None)
valLoader = DataLoader(valDataset,shuffle=False,batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,pin_memory = PIN_MEMORY)
trainLoader = DataLoader(trainDataset,shuffle=False,batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,pin_memory = PIN_MEMORY)

In [9]:
model = Unet(in_channels=3, out_channels=len(project.object_list)).to(DEVICE)
project.load_model(model)
a = model.eval()

In [10]:
x,y,c = next(iter(valLoader))
x.shape

torch.Size([2, 3, 480, 640])

In [11]:
model.eval()
with torch.no_grad():
    out = model(x.to(DEVICE))

In [12]:
fn = os.path.join(project.models_dir,"UNet.onnx")
fn

'/home/kevin/Documents/trackingProjects/faceTrack/models/UNet.onnx'

In [14]:
input_names = [ "input_1" ]
output_names = [ "output1" ]

torch.onnx.export(model, x.to(DEVICE), fn, verbose=False, input_names=input_names, output_names=output_names)

You now have a `onnx` model ready to use in a different framework.